<a href="https://colab.research.google.com/github/Giraud-Pierre/PINN_for_SEDMES/blob/adsorption_exercise/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io

In [ ]:
np.random.seed(0)
tf.random.set_seed(1234)

In [ ]:
class AdsorptionPINN:
  '''PINN model tailored to answer the adsorption exercise'''
  def __init__(x0, Cg0, Cs0, tb, layers, lb, ub):
    '''object constructor (initialize object at creation). Takes the folowing parameters:
    L the length of the bed (m)
    ug the linear gas velocity (m/s)
    epsb the bed porosity (dimensionless)
    c0 the concentration of pollution of the gas (kmol/m3)
    kg mass transfer coefficient of the gas to the particles( m/s)
    Ke the equilibirum constant (dimentionless)
    layers an array containing the number of hidden layers and neurons per layer'''

    #initializing input data
        #initial conditions
    self.x0 = x0
    self.t0 = 0*x0 #at t=0
    self.Cg0 = Cg0
    self.Cs0 = Cs0
        #boundaries
    self.ub = ub
    self.lb = lb
        #lower boundary condition
    self.x_lb = 0*tb + lb[0]
    self.t_lb = tb
        #upper boundary condition
    self.x_ub = 0*tb + ub[0]
    self.t_ub = tb

    #initializing feedforward NN
    self.layers = layers
    self.weights, self.biases = self.initialize_NN(layers)

    #creating tensorflow placeholder
    self.x0_tf = tf.placeholder(tf.float32, shape=[None, self.x0.shape[1]])
    self.t0_tf = tf.placeholder(tf.float32, shape=[None, self.t0.shape[1]])

    self.x_lb_tf = tf.placeholder(tf.float32, shape=[None, self.x_lb.shape[1]])
    self.t_lb_tf = tf.placeholder(tf.float32, shape=[None, self.t_lb.shape[1]])
    
    self.x_ub_tf = tf.placeholder(tf.float32, shape=[None, self.x_ub.shape[1]])
    self.t_ub_tf = tf.placeholder(tf.float32, shape=[None, self.t_ub.shape[1]])



  def initialize_NN(layers):
    '''return initial weights and biases for a feed forward neural network 
    with a given number of layers and neurons per layer'''
    weights = []
    biases = []
    num_layers = len(layers)
    for i in range(num_layers -1):
      #create a set of defaults weights and biases between each layer
      in_dim = layers[i]
      out_dim = layers[i+1]
      xavier_stddev = np.sqrt(2/(in_dim + out_dim))
            #initialize the weights using Xavier initialization to avoid problems such as vanishing or exploding gradients
      W = tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev = xavier_stddev), dtype = tf.float32) 
            #initialize biases at 0
      b = tf.Variable(tf.zeros([1,layers[i+1]], dtype = tf.float32), dtype = tf.float32)
      weights.append(W)
      biases.append(b)
    return weights, biases

In [ ]:
if __name__ == "__main__"
  noise = 0.0 #eventually, can be used to put noise

  # architecture of the feedforward network with 2 inputs being space (x) 
  # and time and 2 outputs being Cg and Cs
  layers = [2, 100, 100, 100, 100, 2] 

  #get data from matlab workspace
  data = scipy.io.loadmat("data/MatlabSimulation.mat") #load the simulation data from matlab

  t = data['t'].flatten()[:,None] # time from simulation
  x = data['x'].flatten()[:,None] # x from simulation
  exact_Cs = data['Cs'] #Cs from simulation, function of x and time
  exact_Cg = data['Cg'] #Cg from simulation, function of x and time

  #Domain bounds
  lb = [0, 0] #lower bondaries [space (m), time (s)]
  ub = [1, 1000] #upper boundaries



  '''Uses collocation points to enforce initial and boundary conditions (can easily be change to get more points)'''
  #number of collocation points per set of collocation points [initial conditions, boundary conditions]
  N0, Nb = [50, 50] 
  ########## Initial conditions: #######################################################################
  idx_x0 = np.random.choice(x.shape[0], N0, replace = False)
  x0 = x[idx_x0,:] #give a random spatial position where to test these initial conditions
  Cg0 = 0*x0 #initial condition for Cg0 at t=0 (here Cg0 = 0)
  Cs0 = 0*x0 #initial condition for Cs0 at t=0 (here Cs0 = 0)
  ########## Boundary conditions: ###################################################################### 
  idx_tb = np.random.choice(t.shape[0], N_b, replace=False)
  tb = t[idx_tb,:] #give a random set of times where to test those boundary conditions

  model = AdsorptionPINN(x0, Cg0, Cs0, tb, layers, lb, ub)


In [ ]:
''' code to get collocation points in the data, might be of use later'''
idx_x1 = np.random.choice(x.shape[0], N1, replace = False) #random space index for the collocation points 
idx_t1 = 3 #time index corresponding to the chosen time
x1 = x[idx_x1,:]
t1 = t[idx_t1,:]
Cs1 = exact_Cs[idx_t1,id_xx]
Cg1 = exact_Cg[idx_t1,idx_x1]